In [ ]:
import pandas as pd
import os
import sys

# Получение пути к текущей рабочей директории (папка А)
script_dir = os.getcwd()

# Путь к родительской папке (папка Б)
parent_dir = os.path.dirname(script_dir)

# Путь к папке prepared_datasets в родительской папке
output_dir = os.path.join(parent_dir, "prepared_datasets")

# Проверка существования папки prepared_datasets
if not os.path.exists(output_dir):
    print(f"Ошибка: папка '{output_dir}' не найдена.")
    sys.exit(1)  # Завершаем выполнение программы с кодом ошибки

# Пути к исходным файлам
file_i = os.path.join(script_dir, "crispr_gene_effect.csv")
file_c = os.path.join(script_dir, "rnai_gene_effect.csv")

# Загрузка данных
scores_i = pd.read_csv(file_i, index_col=0)
scores_c = pd.read_csv(file_c, index_col=0)

# Функция для комбинирования оценок
def combine_scores(scores_i, scores_c, a=0.5):
    genes = scores_i.columns.union(scores_c.columns)
    cells = scores_i.index.union(scores_c.index)
    scores_combined = pd.DataFrame(index=cells, columns=genes, dtype=float)

    for cell in cells:
        for gene in genes:
            value_i = scores_i.at[cell, gene] if cell in scores_i.index and gene in scores_i.columns else None
            value_c = scores_c.at[cell, gene] if cell in scores_c.index and gene in scores_c.columns else None

            if pd.notna(value_i) and pd.notna(value_c):
                scores_combined.at[cell, gene] = (value_i * a) + (value_c * (1 - a))
            elif pd.notna(value_i):
                scores_combined.at[cell, gene] = value_i
            elif pd.notna(value_c):
                scores_combined.at[cell, gene] = value_c

    return scores_combined

# Комбинирование
combined_scores = combine_scores(scores_i, scores_c)
combined_scores.index.name = 'DepMap_ID'

# Сохранение результата
output_file = os.path.join(output_dir, "depscores.csv")
combined_scores.to_csv(output_file)
